In [ ]:
# Code modified from initial realtime report code from Google
# Feature to append api response in google sheet has been done
# This feature will help creating a real time dashboard in looker studio
# We can schedule this script using cloud scheduler
#pip install google-analytics-admin,pip install google-analytics-data

from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    MetricType,
    RunReportRequest,
    RunRealtimeReportRequest
)
import google.oauth2.service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import gspread
from oauth2client.service_account import ServiceAccountCredentials

client = BetaAnalyticsDataClient.from_service_account_json('path_to_your_secret_key_json_file')

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'path_to_your_secret_key_json_file'
)

client_1 = gspread.authorize(credentials)

def print_run_report_response(response,sheet):
    """Prints results of a runReport call."""
    print(f"{response.row_count} rows received")
    for dimensionHeader in response.dimension_headers:
        print(f"Dimension header name: {dimensionHeader.name}")
    for metricHeader in response.metric_headers:
        metric_type = MetricType(metricHeader.type_).name
        print(f"Metric header name: {metricHeader.name} ({metric_type})")


    print("Report result & beginning of data ingestion:")
    cell_list = sheet.range("A1:AA1000")
    next_row = 1
    for rowIdx, row in enumerate(response.rows):
        print(f"\nRow {rowIdx}")
        for i, dimension_value in enumerate(row.dimension_values):
            dimension_name = response.dimension_headers[i].name
            print(f"{dimension_name}: {dimension_value.value}")
            value = dimension_value.value
            column_index = 1
            sheet.update_cell(next_row,column_index,value)

        for i, metric_value in enumerate(row.metric_values):
            metric_name = response.metric_headers[i].name
            print(f"{metric_name}: {metric_value.value}")
            value = metric_value.value
            column_index = 2
            sheet.update_cell(next_row,column_index,value)
        next_row += 1

def run_sample():
    property_id = "your_property_id"
    run_realtime_report(property_id)


def run_realtime_report(property_id="your_property_id"):
    """Runs a realtime report on a Google Analytics 4 property."""
    request = RunRealtimeReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name="eventName")],
        metrics=[Metric(name="eventCount")],
    )
    response = client.run_realtime_report(request)
    sheet = client_1.open("your_sheet_name").sheet1
    sheet.clear()
    print_run_report_response(response,sheet)

if __name__ == "__main__":
    run_sample()